In [ ]:
import os
from random import shuffle
import time
import json
import torch
import hashlib
import argparse
import numpy as np
import pandas as pd
import torch.nn as nn

In [ ]:
from os import listdir
from torch import optim
from copy import deepcopy
from os.path import isfile, join
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import WeightedRandomSampler

In [ ]:
# Preliminaries
TRAIN_DATA_DIR = './JYsplit/train'
VAL_DATA_DIR = './JYsplit/val'
TEST_DATA_DIR = './JYsplit/test'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Preprocess data
transform_train = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
assert os.path.exists(TRAIN_DATA_DIR)
train_dataset = ImageFolder(root=TRAIN_DATA_DIR, transform=transform_train)
val_dataset = ImageFolder(root=VAL_DATA_DIR, transform=transform_train)
test_dataset = ImageFolder(root=TEST_DATA_DIR, transform=transform_train)

print(train_dataset.class_to_idx)

partition = {'train':train_dataset, 'val':val_dataset, 'test':test_dataset}

In [ ]:
# plt.figure(figsize=(15,8))
# sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution(train_dataset)]).melt(),
#             x = "variable", y="value", hue="variable").set_title('Images Class Distribution')

In [ ]:
idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}

def get_class_distribution(dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    
    for element in dataset_obj:
        y_lbl = element[1]
        y_lbl = idx2class[y_lbl]
        count_dict[y_lbl] += 1
            
    return count_dict

print("Distribution of classes: \n", get_class_distribution(train_dataset))

In [ ]:
target_list = torch.tensor(train_dataset.targets)

class_count = [i for i in get_class_distribution(train_dataset).values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
class_weights

In [ ]:
class_weights_all = class_weights[target_list]
class_weights_all

In [ ]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

In [ ]:
# VGG16 Model 
class VGG16(nn.Module):
    def __init__(self, num_classes, in_channels, out_channels, act, dropout, use_bn):
        super(VGG16, self).__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.act = act
        self.dropout = dropout
        self.use_bn = use_bn

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(num_features = 64),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(num_features = 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        ) 

        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )            
        
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )            

        self.fc1 = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(512*7*7, 4096),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(4096, 4096),
            nn.ReLU()
        )

        self.fc3 = nn.Sequential(
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)    
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.view(out.size(0), -1) # 배치사이즈 dimension은 유지하고 나머지 부분은 쭉 펴진 것
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
# Train function 
def train(model, partition, optimizer, criterion, args):
    train_loader = torch.utils.data.DataLoader(
        partition['train'],
        batch_size = args.train_batch_size,
        sampler = weighted_sampler,
        shuffle = False,
        num_workers = 2)
    
    model.train()
    correct = 0
    total = 0
    train_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_loader)
    train_acc = 100 * correct / total

    # model 반환 안해주면 학습시킨 model이 아닌 아무것도 안한 처음 모델로 validation 학습시키는 것
    return model, train_loss, train_acc

In [ ]:
# Validation function
def validate(model, partition, criterion, args):
    val_loader = torch.utils.data.DataLoader(
                    partition['val'],
                    batch_size = args.test_batch_size,
                    shuffle = False,
                    num_workers = 2)
    
    model.eval()
    
    correct = 0
    total = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(val_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total

    return val_loss, val_acc

In [ ]:
# Test function
def test(model, partition, args):
    test_loader = torch.utils.data.DataLoader(partition['test'],
                                                batch_size = args.test_batch_size,
                                                shuffle = False,
                                                num_workers = 2)
    
    model.eval()
    
    correct = 0
    total = 0

    with torch.no_grad():
        for i, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total

    return test_acc

In [ ]:
# Experiment function
def experiment(partition, args):
    model = VGG16(num_classes = args.num_classes, 
                    in_channels = args.in_channels, 
                    out_channels = args.out_channels,
                    act = args.act,
                    dropout = args.dropout,
                    use_bn = args.use_bn)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr = args.lr, weight_decay = args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr = args.lr, weight_decay = args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr = args.lr, weight_decay = args.l2)
    elif args.optim == 'AdamW':
        optimizer = optim.Adam(model.parameters(), lr = args.lr, weight_decay = args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    # =========== List for epoch-wise data ============= #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    # =========== Experiment ============= #
    for epoch in range(args.num_epochs):
        ts = time.time()
        model, train_loss, train_acc = train(model, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(model, partition, criterion, args)
        te = time.time()

        # ========== Add Epoch Data ============ #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)


        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch+1, train_acc, val_acc, train_loss, val_loss, te-ts))
        
        # save model for each 2 epochs
        if (epoch+1) % 2 == 0:
            os.makedirs(f'./models/weightedsampler', exist_ok=True)
            torch.save(model.state_dict(),
                       f'./models/weightedsampler/weighted_{epoch+1}.ckpt')
        
    test_acc = test(model, partition, args)

    # ======== Add Result to Dictionary ========== #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc

    # vars로 감싸면 dictionary가 됨
    return vars(args), result

In [ ]:
def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['num_epochs']
    del setting['test_batch_size']

#    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
#    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    filename = f'./results/weighted_{args.num_epochs}.json'
    result.update(setting) # dictionary와 dictionary 합치기
    with open(filename, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [ ]:
# ======== Random Seed Initialization ======= #
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "bs_64_BN_AdamW_lr_0.0001_vgg16"

# ========= Model Capacity ============= #
args.in_channels = 3
args.out_channels = 64
args.num_groups = 4
args.act = 'relu'

# ========== Regularization ============ #
args.dropout = 0.5
args.use_bn = True
args.l2 = 0.00001

# ========== Optimizer & Training ========= #
args.optim = 'AdamW'
args.num_classes = 20
args.lr = 0.0001
args.num_epochs = 70

args.train_batch_size = 64
args.test_batch_size = 32

# ========= Experiment Variable ========== #
name_var1 = 'num_epochs'
name_var2 = 'lr'
list_var1 = [args.num_epochs]
list_var2 = [args.lr]

for var1 in list_var1:
  for var2 in list_var2:
    setattr(args, name_var1, var1) # args.name_var1 = var1 이라는 문법과 동일
    setattr(args, name_var2, var2)
    print(args)

    setting, result = experiment(partition, deepcopy(args))
    save_exp_result(setting, result)